# Assignment #2

# In This assignment you are asked to read a data which include 48505 articles (Documents). Then fint the most similar documents using Locality Sensitive Hashing. Follow the lecture covering this topic step by step.

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk import ngrams
import string
from scipy.sparse import csr_matrix
import copy

In [3]:
num_frequent_grams = 10000
num_hash_functions = 200
num_buckets = 10000
num_bands = 20

In [ ]:
! pip install nltk

## 1. Data is available in Json format and you need to read it. 'https://www.ux.uis.no/~vsetty/data/assignment2_aricles.json' (5 points)

In [4]:
# 1. Read Data
raw_data = pd.read_json('https://www.ux.uis.no/~vsetty/data/assignment2_aricles.json')
df = raw_data.copy(deep = True)
df.head()

,article_id,Title,Content
0,0,Tikcro enters into research and license agreem...,Tikcro enters into research and license agreem...
1,1,Facebook Friend Request Nearly Cost One North ...,A North Carolina woman is trying to warn other...
2,2,Amlin plc UK Regulatory Announcement: Total Vo...,LONDON--(BUSINESS WIRE)--\n\nAMLIN plc\n\nTOTA...
3,3,Khaleda asks for security,Khaleda asks for security\n\n\n\nBNP Chairpers...
4,4,Liberian Health Clinics Reopen Slowly with Ren...,Liberian Health Clinics Reopen Slowly with Ren...


## 2. Shingle the documents (10 points)
### Tips:
* Use string package to cleanup the articles e.g, str.maketrans('', '', string.
punctuation)
* It is better to convert text to lower case that way you get fewer n-grams
* apply ngrams(x.split(), n) using ngrams from nltk on the content + title for computing n-grams, for this data n = 2 is suffcient
  * You can use n-gram at word level for this task
  * try with different n-gram values 
  * You can use ngrams from nltk for this

In [5]:
string.punctuation + "”“’"

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~”“’'

In [6]:
# 2. Shingle the Documents (cleaning)
nltk.download('punkt')
nltk.download('stopwords')

number_of_articles = df.shape[0]
stop_words = set(nltk.corpus.stopwords.words('english'))

for i in ["Title", "Content"]:
    # Convert to Lowercase and replace newline with space
    df[i] = df[i].apply(lambda x: x.lower().replace('\n', ' '))
    # Remove special characters and Numbers
    df[i] = df[i].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + "”“’0123456789")))
    # Remove stop words so that we don't include them in ngrams
    df[i] = df[i].apply(lambda x: ' '.join([w for w in nltk.tokenize.word_tokenize(x) if w not in stop_words]))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rehman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rehman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Calculate ngrams
df["ngrams-2"] = (df["Title"] + df["Content"]).apply(lambda x: list(ngrams(x.split(), 2)))

In [8]:
df.head()

,article_id,Title,Content,ngrams-2
0,0,tikcro enters research license agreement yeda,tikcro enters research license agreement yeda ...,"[(tikcro, enters), (enters, research), (resear..."
1,1,facebook friend request nearly cost one north ...,north carolina woman trying warn others facebo...,"[(facebook, friend), (friend, request), (reque..."
2,2,amlin plc uk regulatory announcement total vot...,londonbusiness wire amlin plc total voting rig...,"[(amlin, plc), (plc, uk), (uk, regulatory), (r..."
3,3,khaleda asks security,khaleda asks security bnp chairperson khaleda ...,"[(khaleda, asks), (asks, securitykhaleda), (se..."
4,4,liberian health clinics reopen slowly renewed ...,liberian health clinics reopen slowly renewed ...,"[(liberian, health), (health, clinics), (clini..."


## 3. Convert n-grams into binary vector representation for each document. You can do some optimzations if the matrix is too big. (10 points)
* For example,

  * Select top 10000 most frequent n-grams.
  * You may also try smaller values of n (like 2 or 3) which result in fewer n-grams.
  * Finally, you can also try sparse matrix representation. Like csr_matrix from scipy.sparse. It works even with full vocabulary.
    * Given a list of n-grams for each document, see how to builid a sparse matrix here https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

In [9]:
from collections import Counter

In [10]:
def getFrequentNgrams(articles):
    # Your code Here
    # Select most frequent n-grams
    ngrams = []
    for i in articles["ngrams-2"]:
        for j in i:
            ngrams.append(j)
    return [x[0] for x in Counter(ngrams).most_common(num_frequent_grams)]

In [ ]:
# Code from https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
def getBinaryMatrix(docs):
    frequent_ngrams = getFrequentNgrams(docs)
    # We need to only choose ngrams that are among the frequent ones
    frequent_ngrams_as_string_list = []
    for i in range(0, len(frequent_ngrams)):
        tple = frequent_ngrams[i]
        frequent_ngrams_as_string_list.append(f'{tple[0]} {tple[1]}')
    all_ngrams_as_strings = docs["ngrams-2"].copy(deep=True).apply(lambda x: [f'{i[0]} {i[1]}' for i in x])
    docs = []
    for i in all_ngrams_as_strings:
        ngrams_in_frequent = []
        for j in i:
            if j in frequent_ngrams_as_string_list:
                ngrams_in_frequent.append(j)
        docs.append(ngrams_in_frequent)
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in docs:
        for term in d:
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    matrix = csr_matrix((data, indices, indptr), dtype=int).toarray()
    # I don't know why but I am getting numbers bigger than 1 from csr_matrix
    matrix[matrix > 1] = 1
    return matrix

In [11]:
def getBinaryMatrix(docs):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for doc in docs:
        for word in doc:
            index = vocabulary.setdefault(word, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int).toarray()

In [12]:
freq_ngrams = getFrequentNgrams(df)

In [13]:
list_freq_ngrams = list(map(lambda x: f'{x[0]} {x[1]}', freq_ngrams)) # list of all ngrams, in the format of 'word1 word2'

# format the ngrams to be in the format of 'word1 word2'
# get ngrams that are in the top 10000 most frequent ngrams
docs = df["ngrams-2"].copy(deep=True)\
        .apply(lambda x: [f'{i[0]} {i[1]}' for i in x]) \
        .apply(lambda row: np.intersect1d(row, list_freq_ngrams))

In [14]:
docs

0        [research development, st december, today anno...
1        [abc news, could cost, daily basis, didnt know...
2                            [total number, voting rights]
3        [according news, court hearing, former prime, ...
4        [blood pressure, didnt even, dont know, dozen ...
                               ...                        
48500    [available purchase, dozen people, general man...
48501    [around world, could use, looks like, making s...
48502    [former head, health problems, immediately ava...
48503    [country many, daily times, make one, many eve...
48504    [authorities said, december pm, early wednesda...
Name: ngrams-2, Length: 48505, dtype: object

In [15]:
binary_matrix = getBinaryMatrix(docs)

In [16]:
binary_matrix = binary_matrix.T

In [17]:
print(binary_matrix.shape)
print(binary_matrix[:10])

(10000, 48505)
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


## 4. We need hash function that maps integers 0, 1, . . . , k − 1 to bucket numbers 0 through k − 1. It might be impossible to avoid collisions but as long as the collions are too many it won't matter much. (10 points)
* The simplest would be using the builtin hash() function, it can be for example, hash(rownumber) % Numberofbuckets
* You can generate several of these hash functions by xoring a random integer (hash(rownumber)^randint) % Numberofbuckets
* It can also be a as simple as (rownumber * randint) % Numberofbuckets

In [ ]:
def getHashFunctionValues(numrows, numhashfunctions):
    # Your code Here
    # return a matrix with hash values
    # Each column represents a random permutation of row numbers
    hash_matrix = []
    for i in range(numrows):
        temp = []
        for j in range(numhashfunctions):
            temp.append(np.random.randint(0, numrows + 1))
        hash_matrix.append(temp)
    return np.array(hash_matrix)

In [18]:
def getHashFunctionValues(numrows, numhashfunctions):
    hash_matrix = np.empty((numrows, numhashfunctions))
    for i in range(numrows):
        r = np.random.randint(2^32-1)
        hash_matrix[i] = np.array([hash(x)^r for x in range(numhashfunctions)])
    return hash_matrix

In [19]:
hash_matrix = getHashFunctionValues(binary_matrix.shape[0], num_hash_functions)

In [20]:
hash_matrix = hash_matrix.T

In [21]:
hash_matrix.shape

(200, 10000)

In [22]:
binary_matrix.shape

(10000, 48505)

## 5. Compute minhash following the faster algorithm from the lecture (10 points)

In [ ]:
def getMinHashSignatureMatrix(binary_matrix, hash_val_matrix):
    # return minhash signature matrix
    signature_matrix = np.ones((binary_matrix.shape[1], hash_val_matrix.shape[0])) * np.inf
    print(signature_matrix.shape)
    for i in range(binary_matrix.shape[1]):
        for j in range(hash_val_matrix.shape[0]):
            index_comparison = (np.where(binary_matrix[:, i] == 1))[0]
            if len(hash_val_matrix[j, index_comparison]) != 0:
                signature_matrix[i, j] = min(hash_val_matrix[j, index_comparison])
    return signature_matrix

In [23]:
def getMinHashSignatureMatrix(binary_matrix, hash_val_matrix):
    signature_matrix = np.ones((hash_val_matrix.shape[0],binary_matrix.shape[1]))*np.inf

    for i in np.arange(binary_matrix.shape[0]):
        for j in np.where(binary_matrix[i] == 1)[0]:
            for h in np.arange(hash_val_matrix.shape[0]):
                if signature_matrix[h][j] == np.inf or signature_matrix[h][j] > hash_val_matrix[h][i]:
                    signature_matrix[h][j] = hash_val_matrix[h][i]
    
    return signature_matrix

In [24]:
signature_matrix = getMinHashSignatureMatrix(binary_matrix, hash_matrix)

In [28]:
signature_matrix = signature_matrix.T

In [29]:
signature_matrix.shape

(200, 48505)

In [30]:
signature_matrix

array([[  2.,   7.,  10., ...,   1.,   0.,   3.],
       [  3.,   6.,  11., ...,   0.,   1.,   2.],
       [  0.,   5.,   8., ...,   3.,   2.,   1.],
       ...,
       [193., 194., 207., ..., 192., 197., 198.],
       [192., 193., 204., ..., 195., 198., 197.],
       [193., 192., 205., ..., 194., 199., 196.]])

## 6. Hash signature bands into buckets. Find a way to combine all the  signature values in a band and hash them into a number of buckets ususally very high. (10 points)
* Easiest way is to add all the signature values in the bucket and use a similar hash function like before
* You should use the same hash function for all bands. And all documents ending up in same bucket for at least one band are considered as candidate pairs.

In [ ]:
def getLSH(signature_matrix, num_bands, num_buckets):
    # return lsh buckets or hash table
    lsh = {}
    
    rows_per_band = int(signature_matrix.shape[0] / num_bands)
    bands = np.split(signature_matrix, rows_per_band)
    
    # I don't understand how can I limit number of buckets?
    for i in range(len(bands)):
        band = bands[i]
        for j in range(0, band.shape[1]):
            hashed = hash("".join([str(x) for x in band[:, j]]))
            if hashed in lsh:
                lsh[hashed].append(j)
            else:
                lsh[hashed] = [j]    
    return lsh

In [31]:
def getLSH(signature_matrix, num_bands, num_buckets):
    lsh = {}
    r = int(signature_matrix.shape[0]/num_bands)
    for i in range(num_bands):
        rand_int = np.random.randint(2^32-1)
        for j in range(signature_matrix.shape[1]):
            hash_val = hash(tuple(signature_matrix[i*r:(i+1)*r, j]))^rand_int
            if hash_val not in lsh:
                lsh[hash_val] = set()
            lsh[hash_val].add(j)
    return lsh

In [32]:
lsh = getLSH(signature_matrix, num_bands, num_buckets)

## 7. Tune parameters to make sure the threshold is appropriate. (10 points)
* plot the probability of two similar items falling in same bucket for different threshold values

In [33]:
s = 0.7

## 8. Choose the best parameters and get nearest neighbors of each articles (20 points)
* Jaccard Similarity
* convert hash table into dictionary of article ids and its other articles that hashed in at least 1 same bucket

In [ ]:
def getJaccardSimilarityScore(C1, C2):
    intersection = len(list(set(C1).intersection(C2)))
    union = len(C1) + len(C2)
    score = float(intersection / union)
    return score

## 9. Write the nearest neibhors of each document to submissions.csv (comma separated, first column is the current document followed by a list of nearest neighbors) file and get the score (10 points)

In [34]:
# convert hash table into dictionary of article ids and its other articles that hashed in at least 1 same bucket
nearest_neighbors = {}
for _, idx in lsh.items():
    for i in idx:
        if i not in nearest_neighbors:
            nearest_neighbors[i] = set()
        nearest_neighbors[i].update([x for x in idx if x != i])

In [36]:
# convert hash table into dictionary of article ids and its other articles that hashed in at least 1 same bucket
n_copy = {}
for _, idx in lsh.items():
    for i in idx:
        if i not in n_copy:
            n_copy[i] = set()
        n_copy[i].update([x for x in idx if x != i])

MemoryError: 

In [35]:
# Remove the neighbors in same buckets but have similarity score < threshold s
submission_id = []
submission_nid = []
scores = []
for article_id, neighbor_ids in n_copy.items():
    print(neighbor_ids)
    for nid in neighbor_ids:
        score = getJaccardSimilarityScore(binary_matrix[:,article_id], binary_matrix[:,nid])
        print(score)
        if score < s:
            nearest_neighbors[article_id].remove(nid) 
        else:
            # add to submission result
            submission_id.append(article_id)
            submission_nid.append(nid)
            scores.append(score)

MemoryError: 

In [ ]:
data = pd.DataFrame()
data['article_id'] = submission_id
data['neighbor_id'] = submission_nid
data['JaccardScore'] = scores
data.sort_values(by=['article_id', 'neighbor_id'], inplace=True)

In [ ]:
data.head(100)

In [ ]:
submission_id

## 10. Write a report + notebook + submission file in a zip file (5 points)